# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.head()


,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date
0,ushuaia,AR,47.86,66,75,25.32,-54.8000,-68.3000,1643564796
1,puerto ayora,EC,82.36,71,100,8.81,-0.7393,-90.3518,1643564797
2,bluff,NZ,58.80,92,87,3.58,-46.6000,168.3333,1643564798
3,ahuimanu,US,73.45,71,75,1.01,21.4447,-157.8378,1643564800
4,ahipara,NZ,60.84,91,100,7.87,-35.1667,173.1667,1643564801


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"]


In [4]:
# Create a humidity Heatlamp layer and plot heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
vacay_cities = cities_df.loc[(cities_df["Max Temp"] <=85) & (cities_df["Max Temp"] >=75) & (cities_df["Humidity"] <=50) & 
                             (cities_df["Cloudiness"] <=50)]

vacay_cities
#wind speed <=10 if we want to narrow it down more

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date
276,volcan,PA,77.11,40,21,2.08,8.7667,-82.6333,1643565182
283,rocha,UY,82.81,42,6,9.57,-34.4833,-54.3333,1643565194
306,illela,NG,77.32,11,2,12.30,13.7292,5.2975,1643565227
328,ixtapa,MX,78.03,50,0,2.30,20.7000,-105.2000,1643565256
524,neuquen,AR,78.69,25,20,12.66,-38.9516,-68.0591,1643565532
540,maldonado,UY,84.22,42,2,11.01,-34.6667,-54.9167,1643565558


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
# set up new data frame hotel_df and add additional column for Hotel Name

hotel_df = vacay_cities
hotel_df.reset_index(drop=True, inplace=True)

hotel_df['Hotel Name'] = ""

hotel_df

/var/folders/6x/83dn3sr912jd1wjj53dxy4140000gn/T/ipykernel_59064/3690828079.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date,Hotel Name
0,volcan,PA,77.11,40,21,2.08,8.7667,-82.6333,1643565182,
1,rocha,UY,82.81,42,6,9.57,-34.4833,-54.3333,1643565194,
2,illela,NG,77.32,11,2,12.30,13.7292,5.2975,1643565227,
3,ixtapa,MX,78.03,50,0,2.30,20.7000,-105.2000,1643565256,
4,neuquen,AR,78.69,25,20,12.66,-38.9516,-68.0591,1643565532,
5,maldonado,UY,84.22,42,2,11.01,-34.6667,-54.9167,1643565558,


In [29]:
# find the closest hotels within 5000 meters to each city

# set up a parameters dictionary
#params = {
#    "location": coordinates,
#    "keyword": "hotel",
#    "radius": 5000,
#    "type": "lodging",
#   "key": g_key
#}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

   
#iterate through the rows of the df to run a request for the given parameters
for index, row in hotel_df.iterrows():
    
    coordinates = f"{row['Latitude']},{row['Longitude']}"
    params = {"location": coordinates, "radius": 5000,"keyword": "hotel", "type": "lodging", "key": g_key}
    response = requests.get(base_url, params=params).json()
    #print(response)
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        print(response['results'][0]['name'])
    except(KeyError, IndexError):
        print('Missing Field. Skipping...')



Hotel Dos Rios
Hotel Arrarte
Missing Field. Skipping...
Flamingo Vallarta Hotel & Marina
Neuquen Tower Hotel
Missing Field. Skipping...


In [27]:
# Store results for the Hotel into the data frame.
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df

,City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Latitude,Longitude,Date,Hotel Name
0,volcan,PA,77.11,40,21,2.08,8.7667,-82.6333,1643565182,Hotel Dos Rios
1,rocha,UY,82.81,42,6,9.57,-34.4833,-54.3333,1643565194,Hotel Trocadero
2,illela,NG,77.32,11,2,12.30,13.7292,5.2975,1643565227,
3,ixtapa,MX,78.03,50,0,2.30,20.7000,-105.2000,1643565256,Flamingo Vallarta Hotel & Marina
4,neuquen,AR,78.69,25,20,12.66,-38.9516,-68.0591,1643565532,Nuevo Hotel el Cortijo
5,maldonado,UY,84.22,42,2,11.01,-34.6667,-54.9167,1643565558,


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))